In [ ]:
# Random Imports
import os
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

# Regular schema dictates that we put DATAPATH
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ['WANDB_NOTEBOOK_NAME'] = 'train.ipynb'

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

## Define said jobs

In [ ]:
%%yaml default_cfg 

experiment:
  seed: 42
    
dataloader:
  batch_size: 1 
  num_workers: 1
  pin_memory: False 

optim: 
  _class: torch.optim.Adam
  lr: 3.0e-4
  weight_decay: 0.0 
  
train:
  epochs: 1000 
  eval_freq: 10 
  augmentations: None
  pretrained_dir: None 

log:
  checkpoint_freq: 5 
  root: '?'
  metrics:
    dice_score:
      _fn: ionpy.metrics.dice_score
      from_logits: True
      batch_reduction: 'mean' 
      ignore_empty_labels: '?' 
      ignore_index: '?' # Ignore background class when reporting.

######################
# Cross-Entropy Loss #
######################
loss_func: 
  _class: ionpy.loss.PixelCELoss
  from_logits: True
  batch_reduction: 'mean' 
  
#############
# Dice Loss #
#############
# loss_func: 
#   _class: ionpy.loss.SoftDiceLoss
#   from_logits: True
#   batch_reduction: 'mean' 
#   ignore_index: '?'
#   ignore_empty_labels: '?' 

In [ ]:
%%yaml model_cfg  

model:
  _class: ese.experiment.models.UNet
  filters: '?'
  convs_per_block: 3

In [ ]:
%%yaml dataset_cfg 

#######
# WMH #
#######
data:
  _class: ese.experiment.datasets.WMH
  axis: 0
  task: Amsterdam
  slicing: dense 
  annotator: observer_o12
  num_slices: 1
  in_channels: 1
  out_channels: 2 
  version: 0.2
  preload: False 

##################
# OASIS 4-Labels #
##################
# data:
#   _class: ese.experiment.datasets.OASIS
#   axis: 0
#   label_set: label4
#   slicing: central 
#   num_slices: 1
#   in_channels: 1
#   out_channels: 5 
#   central_width: 32
#   version: 0.1
#   preload: False 

###################
# OASIS 35-Labels #
###################
# data:
#   _class: ese.experiment.datasets.OASIS
#   axis: 0
#   label_set: label35
#   slicing: central 
#   num_slices: 1
#   in_channels: 1
#   out_channels: 36 
#   central_width: 32
#   version: 0.1
#   preload: False 

##############
# CityScapes #
##############
# data:
#   _class: ese.experiment.datasets.CityScapes
#   in_channels: 3
#   out_channels: 35

##############
# OxfordPets #
##############
# data:
#   _class: ese.experiment.datasets.OxfordPets
#   preload: True
#   version: 0.1
#   num_classes: all 
#   in_channels: 3
#   out_channels: 38 # 37 + 1 (background)

#####################
# Binary OxfordPets #
#####################
# data:
#   _class: ese.experiment.datasets.BinaryPets
#   preload: True
#   version: 0.1
#   in_channels: 3
#   out_channels: 2

In [ ]:
%%yaml lite_aug_cfg

- RandomAffine:
    p: 0.5
    degrees: [0, 360]
    translate: [0, 0.2]
    scale: [0.8, 1.1]
- RandomVariableElasticTransform:
    p: 0.75
    alpha: [1, 2] 
    sigma: [8, 10]
- RandomHorizontalFlip:
    p: 0.5
- RandomVerticalFlip:
    p: 0.5

In [ ]:
# %%yaml standard_aug_cfg 

# - RandomCrop:
#     _class: ese.experiment.augmentation.transforms.RandomCropSegmentation
#     size: [256, 256]

In [ ]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.experiment.callbacks.ShowPredictions
  epoch:
    - ese.experiment.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: dice_score
        phase: val

## Debug Station

In [ ]:
# Setup direcrtories
train_root = '/storage/vbutoi/scratch/ESE/training'

# WMH CONFIG
exp_name = '11_20_23_WMH_CrossEntropy'
mod_filters = [64, 64, 64, 64, 64]
dl_bs = 8

# BINARY PETS CONFIG
# exp_name = '11_20_23_BinaryPets_CrossEntropy'
# mod_filters = [64, 64, 64, 64, 64]
# dl_bs = 8

# OASIS 4-Label CONFIG
# exp_name = '11_20_23_OASIS4_CrossEntropy'
# mod_filters = [64, 64, 64, 64, 64]
# dl_bs = 8

# OASIS 35-Label CONFIG
# exp_name = '11_20_23_OASIS35_CrossEntropy'
# mod_filters = [64, 64, 64, 64, 64]
# dl_bs = 8

# CityScapes CONFIG
# exp_name = '11_20_23_CityScapes_CrossEntropy'
# mod_filters = [64, 64, 64, 64, 64]
# dl_bs = 8

# Setup the root.
exp_root = f'{train_root}/{exp_name}'

# Create the ablation options
option_set = [
    {
        'log.root': [exp_root],
        'log.metrics.dice_score.ignore_empty_labels': [False], # Set False for WMH, True otherwise.
        'log.metrics.dice_score.ignore_index': [0], # Set 0 for WMH, otherwise is a choice.
        # 'loss_func.ignore_empty_labels': [False], # Set False for WMH, True otherwise. USE FOR DICE
        # 'loss_func.ignore_index': [0], # Set 0 for WMH, otherwise is a choice. USE FOR DICE
        'dataloader.batch_size': [dl_bs],
        'model.filters': [mod_filters]
    },
]

In [ ]:
from ese.scripts.utils import get_option_product
from ionpy.util import Config

# Assemble base config
base_cfg = Config(default_cfg).update([model_cfg, dataset_cfg, callbacks_cfg])

# Get the configs
cfgs = get_option_product(exp_name, option_set, base_cfg)

In [ ]:
len(cfgs)

## Running Jobs

In [ ]:
# FOR DEBUGGING
from ese.experiment.experiment import run_ese_exp, CalibrationExperiment

run_ese_exp(
    config=cfgs[0], 
    experiment_class=CalibrationExperiment,
    gpu='3',
    run_name='debug',
    show_examples=True,
    wandb=False
)

In [ ]:
# FOR SUBMISSION
# from ese.experiment.experiment import run_ese_exp, CalibrationExperiment

# submit_ese_exps(
#     exp_root=exp_root,
#     experiment_class=run_ese_exp,
#     config_list=cfgs,
#     available_gpus=['0'],
#     wandb=True
# )